In [1]:
import tensorflow as tf
import pandas as pd
trainFile = "train_2016_v2.csv"
# trainColumns = ['parcelid','logerror','transactiondate']
df_train = pd.read_csv(trainFile, header = 0, skipinitialspace=True, engine="python")

In [4]:
propertiesFile = "properties_2016.csv"
# propertiesColumns = ['parcelid','airconditioningtypeid','architecturalstyletypeid','basementsqft',
#                      'bathroomcnt','bedroomcnt','buildingclasstypeid','buildingqualitytypeid',
#                      'calculatedbathnbr','decktypeid','finishedfloor1squarefeet','calculatedfinishedsquarefeet',
#                      'finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
#                      'finishedsquarefeet6','fips','fireplacecnt','fullbathcnt','garagecarcnt','garagetotalsqft',
#                      'hashottuborspa','heatingorsystemtypeid','latitude','longitude','lotsizesquarefeet','poolcnt',
#                      'poolsizesum','pooltypeid10','pooltypeid2','pooltypeid7','propertycountylandusecode',
#                      'propertylandusetypeid','propertyzoningdesc','rawcensustractandblock','regionidcity',
#                      'regionidcounty','regionidneighborhood','regionidzip','roomcnt','storytypeid',
#                      'threequarterbathnbr','typeconstructiontypeid','unitcnt','yardbuildingsqft17',
#                      'yardbuildingsqft26','yearbuilt','numberofstories','fireplaceflag','structuretaxvaluedollarcnt',
#                      'taxvaluedollarcnt','assessmentyear','landtaxvaluedollarcnt','taxamount','taxdelinquencyflag',
#                      'taxdelinquencyyear','censustractandblock']

# use float dtype to handle na in pandas

columnDtypes = {'parcelid':int, 'airconditioningtypeid':str, 'architecturalstyletypeid':str, 'basementsqft':float,
                'bathroomcnt':float, 'bedroomcnt':float, 'buildingclasstypeid':str, 'buildingqualitytypeid':str,
                'calculatedbathnbr':float, 'decktypeid':str, 'finishedfloor1squarefeet':float, 'calculatedfinishedsquarefeet':float,
                'finishedsquarefeet12':float, 'finishedsquarefeet13':float, 'finishedsquarefeet15':float, 'finishedsquarefeet50':float,
                'finishedsquarefeet6':float, 'fips':str, 'fireplacecnt':float, 'fullbathcnt':float, 'garagecarcnt':float, 'garagetotalsqft':float,
                'hashottuborspa':str, 'heatingorsystemtypeid':str, 'latitude':float, 'longitude':float, 'lotsizesquarefeet':float, 'poolcnt':float,
                'poolsizesum':float, 'pooltypeid10':str, 'pooltypeid2':str, 'pooltypeid7':str, 'propertycountylandusecode':str,
                'propertylandusetypeid':str, 'propertyzoningdesc':str, 'rawcensustractandblock':str, 'regionidcity':str,
                'regionidcounty':str, 'regionidneighborhood':str, 'regionidzip':str, 'roomcnt':float, 'storytypeid':float,
                'threequarterbathnbr': float, 'typeconstructiontypeid':float, 'unitcnt':float, 'yardbuildingsqft17':float,
                'yardbuildingsqft26':float, 'yearbuilt':float, 'numberofstories': float, 'fireplaceflag':str, 'structuretaxvaluedollarcnt':float,
                'taxvaluedollarcnt': float, 'assessmentyear':float, 'landtaxvaluedollarcnt': float, 'taxamount':float, 'taxdelinquencyflag':str,
                'taxdelinquencyyear': float, 'censustractandblock':float}
df_properties = pd.read_csv(propertiesFile, header = 0, skipinitialspace=True, dtype=columnDtypes, 
                            engine="c")



In [6]:
'''Return the index of the input dataframe where the given column has nan values.
'''
def getIndex(df, colName):
    tmpdf = df[colName].isnull()
    return tmpdf[tmpdf == True].index

In [7]:
inter = pd.merge(df_properties, df_train, how="inner", on=["parcelid"]);
inter.shape

(90275, 60)

In [8]:
inter.isnull().sum(axis=0)

parcelid                            0
airconditioningtypeid           61494
architecturalstyletypeid        90014
basementsqft                    90232
bathroomcnt                         0
bedroomcnt                          0
buildingclasstypeid             90259
buildingqualitytypeid           32911
calculatedbathnbr                1182
decktypeid                      89617
finishedfloor1squarefeet        83419
calculatedfinishedsquarefeet      661
finishedsquarefeet12             4679
finishedsquarefeet13            90242
finishedsquarefeet15            86711
finishedsquarefeet50            83419
finishedsquarefeet6             89854
fips                                0
fireplacecnt                    80668
fullbathcnt                      1182
garagecarcnt                    60338
garagetotalsqft                 60338
hashottuborspa                  87910
heatingorsystemtypeid           34195
latitude                            0
longitude                           0
lotsizesquar

In [9]:
inter.dropna(axis=1, thresh = int(inter.shape[0] * .9), inplace=True);
inter.shape

(90275, 26)

In [10]:
inter.isnull().sum(axis=0)

parcelid                           0
bathroomcnt                        0
bedroomcnt                         0
calculatedbathnbr               1182
calculatedfinishedsquarefeet     661
finishedsquarefeet12            4679
fips                               0
fullbathcnt                     1182
latitude                           0
longitude                          0
propertycountylandusecode          1
propertylandusetypeid              0
rawcensustractandblock             0
regionidcity                    1803
regionidcounty                     0
regionidzip                       35
roomcnt                            0
yearbuilt                        756
structuretaxvaluedollarcnt       380
taxvaluedollarcnt                  1
assessmentyear                     0
landtaxvaluedollarcnt              1
taxamount                          6
censustractandblock              605
logerror                           0
transactiondate                    0
dtype: int64

In [8]:
#inter.fillna(0.0, inplace=True)

In [19]:
#inter.dropna(axis = 1, how='any', inplace=True)
# inter.dropna(axis = 0, how='any', inplace=True)

In [10]:
# inter.set_index("parcelid");

In [11]:
import numpy as np
np.random.seed(1)
datasetSize = inter.shape[0]
trainRatio = .8
trainIndex = set(np.random.choice(datasetSize, int(datasetSize * trainRatio), replace=False))
testIndex = set(range(datasetSize)) - trainIndex
# cast to list to indexing dataframe
trainIndex = list(trainIndex)
testIndex = list(testIndex)

In [12]:
df_trainx = inter.iloc[trainIndex]
trainy = df_trainx['logerror'].astype(float)
df_trainx = df_trainx.drop(['logerror', 'transactiondate'], axis=1)
df_testx = inter.iloc[testIndex]
testy = df_testx['logerror'].astype(float)
df_testx = df_testx.drop(['logerror', 'transactiondate'], axis=1)

df_trainx.set_index('parcelid');
df_testx.set_index('parcelid');

In [13]:
def input_fn(df_trainx, trainy, num_epochs, shuffle=False, batch_size=100, num_threads=5):
    return tf.estimator.inputs.pandas_input_fn(
        x=df_trainx,
        y=trainy,
        batch_size=batch_size,
        num_epochs=num_epochs,
        shuffle=shuffle,
        num_threads=num_threads)

In [14]:
# continous features no bukcet. To-do: use buckets to handle -1s used to fill nas
bathroomcnt = tf.feature_column.numeric_column("bathroomcnt")
bedroomcnt = tf.feature_column.numeric_column("bedroomcnt")
calculatedbathnbr = tf.feature_column.numeric_column('calculatedbathnbr')
calculatedfinishedsquarefeet = tf.feature_column.numeric_column('calculatedfinishedsquarefeet')
finishedsquarefeet12 = tf.feature_column.numeric_column("finishedsquarefeet12")
fullbathcnt = tf.feature_column.numeric_column("fullbathcnt")
latitude = tf.feature_column.numeric_column("latitude")
longitude = tf.feature_column.numeric_column("longitude")
# treat rawcensustractandblock as numerical data
rawcensustractandblock = tf.feature_column.numeric_column("rawcensustractandblock")
censustractandblock = tf.feature_column.numeric_column("censustractandblock")
roomcnt = tf.feature_column.numeric_column("roomcnt")
yearbuilt = tf.feature_column.numeric_column("yearbuilt")
structuretaxvaluedollarcnt = tf.feature_column.numeric_column("structuretaxvaluedollarcnt")
taxvaluedollarcnt = tf.feature_column.numeric_column("taxvaluedollarcnt")
assessmentyear = tf.feature_column.numeric_column("assessmentyear")
landtaxvaluedollarcnt = tf.feature_column.numeric_column("landtaxvaluedollarcnt")
taxamount = tf.feature_column.numeric_column("taxamount")

numeric_features = [bathroomcnt, bedroomcnt, calculatedbathnbr, calculatedfinishedsquarefeet, finishedsquarefeet12,
                    fullbathcnt, latitude, longitude, rawcensustractandblock, censustractandblock, roomcnt,
                    yearbuilt, structuretaxvaluedollarcnt, taxvaluedollarcnt, assessmentyear, landtaxvaluedollarcnt,
                    taxamount]
numeric_features = [bathroomcnt]

In [15]:
# categorical features 
fips = tf.feature_column.categorical_column_with_vocabulary_list(
    "fips", [
        "06037", "06059", "06111"
    ])
propertycountylandusecode = tf.feature_column.categorical_column_with_hash_bucket(
    "propertycountylandusecode", hash_bucket_size=100)
propertylandusetypeid = tf.feature_column.categorical_column_with_hash_bucket(
    "propertylandusetypeid", hash_bucket_size=20)
regionidcity = tf.feature_column.categorical_column_with_hash_bucket(
    "regionidcity", hash_bucket_size=300)
regionidcounty = tf.feature_column.categorical_column_with_vocabulary_list(
    "regionidcounty", [
        "1286", "2061", "3101"
    ])
regionidzip = tf.feature_column.categorical_column_with_hash_bucket(
    "regionidzip", hash_bucket_size=500)
# fireplaceflag = tf.feature_column.categorical_column_with_vocabulary_list(
#     "fireplaceflag", [
#         "1.0", "0.0"
#     ])

# categorical_features = [fips, propertycountylandusecode, propertylandusetypeid, regionidcity, regionidcounty,
#                         regionidzip]

In [16]:
# import os, errno
# dir_name = "test"
# try:
#     model_dir = os.makedirs("./{}".format(dir_name))
# except OSError as e:
#     if e.errno != errno.EEXIST:
#         raise
#     else: 
#         model_dir = "./test"
#         print("directoriy already existed")

In [16]:
base_features = numeric_features
import tempfile
model_dir = tempfile.mkdtemp()
m = tf.estimator.LinearRegressor(
        model_dir=model_dir, feature_columns=base_features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/dx/pz80lsmn42xftdx1wckt0zh00000gn/T/tmp7wjafj2t', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [17]:
m.train(input_fn=input_fn(df_trainx, trainy, num_epochs=None), steps = 10)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Error reported to Coordinator: <class 'SystemError'>, <built-in function TF_Run> returned a result with an error set
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/dx/pz80lsmn42xftdx1wckt0zh00000gn/T/tmp7wjafj2t/model.ckpt.


SystemError: <built-in function TF_Run> returned a result with an error set